In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from IPython.display import clear_output
import pandas as pd
import random

In [3]:
df_urls = pd.read_csv('urls_propiedades_en_venta_san_martin_argenprop.csv', index_col=False)
urls = df_urls['0'].tolist()

In [4]:
%%time

exceptions = []
features_global = []

for url in urls:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    try:
        features_local = {}
        driver.get(url)

        # Extraccion del precio en USD

        try:
            element = driver.find_element(By.CLASS_NAME, 'titlebar__price').text
            element = element.replace('USD', '')
            element = element.replace('.', '')
            element = element.replace(' ', '')
            features_local['precio_USD'] = int(element)
        except:
            pass

        # Categoria de la propiedad (casa, departamento o ph)

        if 'casa' in url:
            features_local['categoria'] = 'casa'
        elif 'ph' in url: 
            features_local['categoria'] = 'ph'
        elif 'departamento' in url:
            features_local['categoria'] = 'departamento'

        # Extraccion de los elementos principales

        try:
            element = driver.find_element(By.CLASS_NAME, 'property-main-features')
            elements = element.find_elements(By.TAG_NAME, 'p')
        except:
            pass

        try:
            features_local[elements[0].text] = elements[1].text
        except:
            pass

        try:
            features_local[elements[2].text] = elements[3].text
        except:
            pass

        try:
            features_local[elements[4].text] = elements[5].text
        except:
            pass

        try:
            features_local[elements[6].text] = elements[7].text
        except:
            pass

        try:
            features_local[elements[8].text] = elements[9].text
        except:
            pass

        # Correcion de elementos que son equivalentes, son inecesarios o hay que formatear.

        try:
            features_local['Baños'] = features_local['Cantidad de baños']
            del features_local['Cantidad de baños']
        except:
            pass

        try:
            features_local['Baños'] = int(float(features_local['Baños']))
        except:
            pass

        try:
            features_local['Dormitorios'] = features_local['Cantidad de dormitorios']
            del features_local['Cantidad de dormitorios']
        except:
            pass

        try:
            features_local['Dormitorios'] = features_local['Dormitorios'].replace('Monoambiente', '0')
            features_local['Dormitorios'] = int(float(features_local['Dormitorios']))
        except:
            pass

        try:
            features_local['Antiguedad'] = features_local['Antigüedad']
            del features_local['Antigüedad']
        except:
            pass

        try:
            features_local['Antiguedad'] = features_local['Antiguedad'].replace('A Estrenar', '0')
            features_local['Antiguedad'] = features_local['Antiguedad'].replace('A estrenar', '0')
            features_local['Antiguedad'] = features_local['Antiguedad'].replace(' años', '')
            features_local['Antiguedad'] = features_local['Antiguedad'].replace(' año', '')
            features_local['Antiguedad'] = features_local['Antiguedad'].replace(',', '.')
            features_local['Antiguedad'] = int(float(features_local['Antiguedad']))
        except:
            pass

        try:
            features_local['Superficie cubierta'] = features_local['Superficie cubierta'].replace('m²', '')
            features_local['Superficie cubierta'] = features_local['Superficie cubierta'].replace(' ', '')
            features_local['Superficie cubierta'] = features_local['Superficie cubierta'].replace(',', '.')
            features_local['Superficie cubierta'] = float(features_local['Superficie cubierta'])
            features_local['Superficie cubierta'] = int(features_local['Superficie cubierta'])
        except:
            pass

        try:
            del features_local['Disposición']
        except:
            pass

        # Extracion de la latitud y la longitud

        try:
            element = driver.find_element(By.CLASS_NAME, 'map-container')
            element = element.find_element(By.TAG_NAME, 'div').get_attribute('data-latitude')
            features_local['latitude'] = float(element.replace(',','.'))
        except:
            pass

        try:
            element = driver.find_element(By.CLASS_NAME, 'map-container')
            element = element.find_element(By.TAG_NAME, 'div').get_attribute('data-longitude')
            features_local['longitude'] = float(element.replace(',','.'))
        except:
            pass

        # Extraccion del vendedor

        try:
            element = driver.find_element(By.CLASS_NAME, 'details')
            element = element.find_element(By.TAG_NAME, 'h6').text
            features_local['seller'] = element
        except:
            pass


        features_global.append(features_local)

        clear_output(wait=True)

        print(f'Propiedad número {urls.index(url) + 1} extraída ({round(((urls.index(url) + 1)/len(urls))*100 , 2)}%)')

        df = pd.DataFrame(features_global)

        df = df.rename(columns={'precio_USD':'price_USD',
                                'categoria':'type',
                                'Superficie cubierta':'covered_surface_m2',
                                'Dormitorios':'bedrooms',
                                'Baños':'bathrooms',
                                'logitud':'longitude',
                                'Antiguedad':'antiquity'})

        df.to_csv('features_propiedades.csv')
        driver.close()
    except:
        exceptions.append(urls.index(url))
        driver.close()

Propiedad número 8100 extraída (100.0%)
CPU times: user 14min 1s, sys: 3min 5s, total: 17min 7s
Wall time: 10h 8min 27s


In [5]:
exceptions

[4822, 5941, 7304]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3726 entries, 0 to 3725
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   price_USD           3511 non-null   float64
 1   type                3726 non-null   object 
 2   covered_surface_m2  3491 non-null   float64
 3   bedrooms            3636 non-null   float64
 4   bathrooms           3418 non-null   float64
 5   antiquity           2886 non-null   float64
 6   latitude            3410 non-null   float64
 7   longitude           3410 non-null   float64
 8   seller              3532 non-null   object 
dtypes: float64(7), object(2)
memory usage: 262.1+ KB


In [7]:
df2 = df
df2

,price_USD,type,covered_surface_m2,bedrooms,bathrooms,antiquity,latitude,longitude,seller
0,105000.0,casa,115.0,2.0,1.0,53.0,-34.552040,-58.567482,Inmobiliaria Viviana Villalobos
1,60000.0,ph,NaN,2.0,1.0,NaN,-34.568157,-58.541866,Alicia Pinasco inmobiliaria
2,63000.0,departamento,55.0,1.0,1.0,0.0,-34.572796,-58.520504,Inmobiliaria Viviana Villalobos
3,69000.0,casa,57.0,1.0,1.0,0.0,-34.572796,-58.520504,Inmobiliaria Viviana Villalobos
4,69000.0,casa,57.0,1.0,1.0,0.0,-34.572796,-58.520504,Inmobiliaria Viviana Villalobos
...,...,...,...,...,...,...,...,...,...
3721,45000.0,departamento,35.0,0.0,1.0,0.0,-34.578136,-58.532368,RE/MAX Buró II
3722,78000.0,departamento,54.0,2.0,1.0,0.0,-34.578068,-58.532350,RE/MAX Buró II
3723,48000.0,departamento,38.0,0.0,1.0,0.0,-34.578102,-58.532450,RE/MAX Buró II
3724,43000.0,departamento,35.0,0.0,1.0,0.0,-34.577976,-58.532475,RE/MAX Buró II


In [8]:
df2.columns

Index(['price_USD', 'type', 'covered_surface_m2', 'bedrooms', 'bathrooms',
       'antiquity', 'latitude', 'longitude', 'seller'],
      dtype='object')

In [9]:
df = pd.read_csv('features_propiedades_4371.csv')
df.drop(axis=1, labels=['Unnamed: 0'], inplace=True)
df

,price_USD,type,covered_surface_m2,bedrooms,bathrooms,latitude,longitude,seller,antiquity
0,NaN,departamento,44.0,1.0,1.0,NaN,NaN,NaN,6.0
1,122000.0,departamento,53.0,1.0,2.0,-34.545467,-58.552177,Altamirano Propiedades,4.0
2,93500.0,departamento,55.0,1.0,1.0,-34.549740,-58.550430,MANCINI PROPIEDADES,0.0
3,69000.0,departamento,42.0,1.0,1.0,-34.543205,-58.552757,FALUGUE PROPIEDADES,1.0
4,150000.0,departamento,85.0,2.0,2.0,-34.548748,-58.556840,Matias Fortini Propiedades,0.0
...,...,...,...,...,...,...,...,...,...
4367,60000.0,ph,NaN,2.0,1.0,-34.568157,-58.541866,Alicia Pinasco inmobiliaria,NaN
4368,63000.0,departamento,55.0,1.0,1.0,-34.572796,-58.520504,Inmobiliaria Viviana Villalobos,0.0
4369,69000.0,casa,57.0,1.0,1.0,-34.572796,-58.520504,Inmobiliaria Viviana Villalobos,0.0
4370,69000.0,casa,57.0,1.0,1.0,-34.572796,-58.520504,Inmobiliaria Viviana Villalobos,0.0


In [10]:
df[df2.columns].append(df2).reset_index(drop = True)

,price_USD,type,covered_surface_m2,bedrooms,bathrooms,antiquity,latitude,longitude,seller
0,NaN,departamento,44.0,1.0,1.0,6.0,NaN,NaN,NaN
1,122000.0,departamento,53.0,1.0,2.0,4.0,-34.545467,-58.552177,Altamirano Propiedades
2,93500.0,departamento,55.0,1.0,1.0,0.0,-34.549740,-58.550430,MANCINI PROPIEDADES
3,69000.0,departamento,42.0,1.0,1.0,1.0,-34.543205,-58.552757,FALUGUE PROPIEDADES
4,150000.0,departamento,85.0,2.0,2.0,0.0,-34.548748,-58.556840,Matias Fortini Propiedades
...,...,...,...,...,...,...,...,...,...
8093,45000.0,departamento,35.0,0.0,1.0,0.0,-34.578136,-58.532368,RE/MAX Buró II
8094,78000.0,departamento,54.0,2.0,1.0,0.0,-34.578068,-58.532350,RE/MAX Buró II
8095,48000.0,departamento,38.0,0.0,1.0,0.0,-34.578102,-58.532450,RE/MAX Buró II
8096,43000.0,departamento,35.0,0.0,1.0,0.0,-34.577976,-58.532475,RE/MAX Buró II


In [12]:
df[df2.columns].append(df2).reset_index(drop = True).to_csv('propiedades.csv')

In [14]:
pd.read_csv('propiedades.csv',index_col=0)

,price_USD,type,covered_surface_m2,bedrooms,bathrooms,antiquity,latitude,longitude,seller
0,NaN,departamento,44.0,1.0,1.0,6.0,NaN,NaN,NaN
1,122000.0,departamento,53.0,1.0,2.0,4.0,-34.545467,-58.552177,Altamirano Propiedades
2,93500.0,departamento,55.0,1.0,1.0,0.0,-34.549740,-58.550430,MANCINI PROPIEDADES
3,69000.0,departamento,42.0,1.0,1.0,1.0,-34.543205,-58.552757,FALUGUE PROPIEDADES
4,150000.0,departamento,85.0,2.0,2.0,0.0,-34.548748,-58.556840,Matias Fortini Propiedades
...,...,...,...,...,...,...,...,...,...
8093,45000.0,departamento,35.0,0.0,1.0,0.0,-34.578136,-58.532368,RE/MAX Buró II
8094,78000.0,departamento,54.0,2.0,1.0,0.0,-34.578068,-58.532350,RE/MAX Buró II
8095,48000.0,departamento,38.0,0.0,1.0,0.0,-34.578102,-58.532450,RE/MAX Buró II
8096,43000.0,departamento,35.0,0.0,1.0,0.0,-34.577976,-58.532475,RE/MAX Buró II
